https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275

In [25]:

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense,Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import numpy as np

In [4]:
data = """
If you can make one heap of all your winnings
And risk it on one turn of pitch-and-toss,
And lose, and start again at your beginnings,
And never breathe a word about your loss:
If you can force your heart and nerve and sinew
To serve your turn long after they are gone,
And so hold on when there is nothing in you
Except the Will which says to them: 'Hold on! '

If you can talk with crowds and keep your virtue,
Or walk with Kings- -nor lose the common touch,
If neither foes nor loving friends can hurt you,
If all men count with you, but none too much:
If you can fill the unforgiving minute
With sixty seconds' worth of distance run,
Yours is the Earth and everything that's in it,
And- -which is more- -you'll be a Man, my son!"""

In [23]:
tokenizer = Tokenizer()
max_sequence_len = 0

def dataset_preparation(data):
    corpus = data.lower().split("\n")
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    input_sequences = []
    
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array([pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')])
    
    predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
    label = ku.to_categorical(label,num_classes=total_words)
    
    return predictors, label, max_sequence_len, total_words

https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714


In [32]:

def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=max_sequence_len -1))
    model.add(LSTM(150, return_sequences = True))
    model.add(Dropout(0.1))
    model.add(Dense(total_words,activation ='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    model.fit(predictors, label, epochs=100, verbose= 1)
    

In [12]:
def generate_text():
    for  j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen = max_sequence_len-1,padding='pre')
        predicted = model.predict_classes(token_list, verbose =0)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [33]:
X, Y, max_len, total_words = dataset_preparation(data)
model = create_model(X, Y, max_len, total_words)

ValueError: Error when checking input: expected embedding_5_input to have 2 dimensions, but got array with shape (1, 126, 128)